In [45]:
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv('../data/process_v1/process_data_4.csv')

In [47]:
del df['unitPrice']

In [48]:
del df['landType']

In [49]:
df.head(3)

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,typeOfRealEstate,frontWidth,endWidth,facade,houseDirection,landSize,price,accessibility,description,time
0,1,2.0,2.0,NaN,NaN,no,thủ đức - tam phú,thủ đức - tam phú - cây keo,thủ đức,hồ chí minh,...,condominium,0.0,NaN,oneSideOpen,NaN,58.0,1.5,notInTheAlley,bán căn chung cư đạt gia. diện tích: 58m². giá...,2023-12-27T00:00:00
1,1,2.0,3.0,NaN,NaN,yes,quận 7 - tân phong,quận 7 - tân phong - tôn dật tiên,quận 7,hồ chí minh,...,condominium,0.0,NaN,threeSideOpen,NaN,150.0,8.5,notInTheAlley,cần bán gấp căn hộ chung cư tại garden plaza p...,2023-12-25T00:00:00
2,1,NaN,NaN,NaN,NaN,yes,nam từ liêm - mễ trì,nam từ liêm - mễ trì - lê quang đạo,nam từ liêm,hà nội,...,condominium,0.0,NaN,oneSideOpen,NaN,88.0,0.0,notInTheAlley,full quỹ căn the matrix one chuyển nhượng và c...,2023-11-06T00:00:00


In [50]:
df['w'] = df['frontWidth']
df['h'] = df['landSize'] / df['w']

In [51]:
df = df.replace(np.inf, np.nan)

In [52]:
df.columns.tolist()

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'numberOfKitchens',
 'numberOfGarages',
 'certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'lat',
 'lon',
 'typeOfRealEstate',
 'frontWidth',
 'endWidth',
 'facade',
 'houseDirection',
 'landSize',
 'price',
 'accessibility',
 'description',
 'time',
 'w',
 'h']

In [53]:
location_df = df[['lat', 'lon']].drop_duplicates()

In [54]:
facility_500_df = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_hn_distance_500_facility_count.csv')
facility_1000_df = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_hn_distance_1000_facility_count.csv')
facility_2000_df = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_hn_distance_2000_facility_count.csv')

In [55]:
def rename_col(dataframe, distance):
    cols = [c for c in dataframe.columns if c  not in ['lat', 'lon']]
    rename_cols = [f'num_of_{col}_in_{distance}m_radius' for col in cols]

    rename_col_dict = dict(zip(cols, rename_cols))
    return dataframe.rename(columns = rename_col_dict)

facility_500_df = rename_col(facility_500_df, 500)
facility_1000_df = rename_col(facility_1000_df, 1000)
facility_2000_df = rename_col(facility_2000_df, 2000)


In [56]:
for facility_df in [facility_500_df, facility_1000_df, facility_2000_df]:
    df = df.merge(facility_df, how='left', on = ['lat', 'lon'])

In [57]:
df

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,num_of_marketplace_in_2000m_radius,num_of_restaurant_in_2000m_radius,num_of_hospital_in_2000m_radius,num_of_school_in_2000m_radius,num_of_kindergarten_in_2000m_radius,num_of_townhall - community_centre_in_2000m_radius,num_of_police_in_2000m_radius,num_of_place_of_worship_in_2000m_radius,num_of_bank_in_2000m_radius,num_of_atm_in_2000m_radius
0,1,2.0,2.0,NaN,NaN,no,thủ đức - tam phú,thủ đức - tam phú - cây keo,thủ đức,hồ chí minh,...,6,4,3,19,2,0,0,20,3,3
1,1,2.0,3.0,NaN,NaN,yes,quận 7 - tân phong,quận 7 - tân phong - tôn dật tiên,quận 7,hồ chí minh,...,1,71,5,26,2,0,4,3,12,20
2,1,NaN,NaN,NaN,NaN,yes,nam từ liêm - mễ trì,nam từ liêm - mễ trì - lê quang đạo,nam từ liêm,hà nội,...,2,13,2,25,5,0,2,7,2,11
3,4,NaN,NaN,NaN,NaN,no,hai bà trưng - minh khai,hai bà trưng - minh khai - minh khai,hai bà trưng,hà nội,...,5,11,6,46,24,0,16,4,13,59
4,1,NaN,NaN,NaN,NaN,yes,suburb_north - bắc sơn,suburb_north - bắc sơn - bắc sơn,suburb_north,hà nội,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170959,5,3.0,4.0,NaN,NaN,yes,cầu giấy - yên hòa,cầu giấy - yên hòa - yên hòa,cầu giấy,hà nội,...,17,117,8,58,24,0,10,24,37,122
170960,1,1.0,1.0,NaN,NaN,no,cầu giấy - trung hòa,cầu giấy - trung hòa - trần duy hưng,cầu giấy,hà nội,...,14,82,8,58,23,0,10,14,39,92
170961,5,4.0,4.0,NaN,NaN,yes,đống đa - hàng bột,đống đa - hàng bột - tôn đức thắng,đống đa,hà nội,...,13,390,22,71,21,0,45,73,105,354
170962,1,NaN,NaN,NaN,NaN,no,tây hồ - phú thượng,tây hồ - phú thượng - an dương vương,tây hồ,hà nội,...,1,9,1,9,0,0,3,4,4,6


In [58]:
num_cols = []
cat_cols = []
for c in df.columns.tolist():
    if df[c].dtype != 'object':
        num_cols.append(c)
    else:
        cat_cols.append(c)


In [59]:
cat_cols

['certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'typeOfRealEstate',
 'facade',
 'houseDirection',
 'accessibility',
 'description',
 'time']

In [60]:
num_cols

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'numberOfKitchens',
 'numberOfGarages',
 'lat',
 'lon',
 'frontWidth',
 'endWidth',
 'landSize',
 'price',
 'w',
 'h',
 'num_of_university_in_500m_radius',
 'num_of_fuel_in_500m_radius',
 'num_of_cafe_in_500m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_parking_entrance_in_500m_radius',
 'num_of_fast_food_in_500m_radius',
 'num_of_marketplace_in_500m_radius',
 'num_of_restaurant_in_500m_radius',
 'num_of_hospital_in_500m_radius',
 'num_of_school_in_500m_radius',
 'num_of_kindergarten_in_500m_radius',
 'num_of_townhall - community_centre_in_500m_radius',
 'num_of_police_in_500m_radius',
 'num_of_place_of_worship_in_500m_radius',
 'num_of_bank_in_500m_radius',
 'num_of_atm_in_500m_radius',
 'num_of_university_in_1000m_radius',
 'num_of_fuel_in_1000m_radius',
 'num_of_cafe_in_1000m_radius',
 'num_of_parking_in_1000m_radius',
 'num_of_parking_entrance_in_1000m_radius',
 'num_of_fast_food_in_1000m_radius',
 'num_of_market

In [61]:
df[
'typeOfRealEstate'
].value_counts()

typeOfRealEstate
privateProperty         77386
townhouse               30638
condominium             29533
privateLand             19695
semiDetachedVilla       11563
shophouse                1500
otherTypesOfProperty      561
resort                     88
Name: count, dtype: int64

In [62]:
certificateOfLandUseRight_list = list(df['certificateOfLandUseRight'].unique())
certificateOfLandUseRight_dict = dict(zip(certificateOfLandUseRight_list, range(len(certificateOfLandUseRight_list))))
df['certificateOfLandUseRight'] = df['certificateOfLandUseRight'].map(certificateOfLandUseRight_dict)

ward_list = list(df['ward'].unique())
ward_dict = dict(zip(ward_list, range(len(ward_list))))
df['ward'] = df['ward'].map(ward_dict)
street_list = list(df['street'].unique())
street_dict = dict(zip(street_list, range(len(street_list))))
df['street'] = df['street'].map(street_dict)
district_list = list(df['district'].unique())
district_dict = dict(zip(district_list, range(len(district_list))))
df['district'] = df['district'].map(district_dict)
city_list = list(df['city'].unique())
city_dict = dict(zip(city_list, range(len(city_list))))
df['city'] = df['city'].map(city_dict)
typeOfRealEstate_list = list(df['typeOfRealEstate'].unique())
typeOfRealEstate_dict = dict(zip(typeOfRealEstate_list, range(len(typeOfRealEstate_list))))
df['typeOfRealEstate'] = df['typeOfRealEstate'].map(typeOfRealEstate_dict)
facade_list = list(df['facade'].unique())
facade_dict = dict(zip(facade_list, range(len(facade_list))))
df['facade'] = df['facade'].map(facade_dict)
houseDirection_list = list(df['houseDirection'].unique())
houseDirection_dict = dict(zip(houseDirection_list, range(len(houseDirection_list))))
df['houseDirection'] = df['houseDirection'].map(houseDirection_dict)
accessibility_list = list(df['accessibility'].unique())
accessibility_dict = dict(zip(accessibility_list, range(len(accessibility_list))))
df['accessibility'] = df['accessibility'].map(accessibility_dict)

In [63]:
accessibility_dict

{'notInTheAlley': 0,
 'fitThreeCars': 1,
 'parkCar': 2,
 'fitOneCarAndOneMotorbike': 3,
 'fitTwoCars': 4,
 'theBottleNeckPoint': 5,
 'narrorRoad': 6}

In [64]:
df1 = pd.read_csv('../feature_repo/src/feature_engineering/population/data/table/hcm_population.csv')
df2 = pd.read_csv('../feature_repo/src/feature_engineering/population/data/table/hn_population.csv')

population_df = pd.concat([
    df1, df2
])

def convert_district_to_num(district):
    if district in ['mê linh',
        'ba vì',
        'phúc thọ',
        'thạch thất',
        'mỹ đức',
        'sơn tây',
        'quốc oai',
        'quốc oai']:
        district = 'suburb_west'
    elif district in ['thanh oai',
'ứng hòa',
'phú xuyên',
'thường tín',
'chương mỹ',]:
        district = 'suburb_south'
    elif district in ['sóc sơn', 'đan phượng']:
        district = 'suburb_north'
    candidate_str = [district, f"quận {district}", f"huyện {district}", f"thị xã {district}"]

    for candidate in candidate_str:
        try:
            return district_dict[candidate]
        except:pass
    print(district)
population_df['district'] = population_df['name'].apply(convert_district_to_num)
population_df

tây dừng
chúc sơn
trâu quỳ
văn điển
vân đình


,administrative_genre,name,acreage,population,density,lat,lon,average_price,num_of_ward,district
0,quận,1,7.7300,205180.0,26.543338,10.775818,106.674663,479.000,10,11.0
1,quận,2,49.7400,168680.0,3.391234,10.780199,106.667239,80.838,11,18.0
2,quận,3,4.9200,196433.0,39.925407,10.779290,106.660524,290.600,14,32.0
3,quận,4,4.1800,203060.0,48.578947,10.760791,106.683567,164.400,15,39.0
4,quận,5,4.2700,1871510.0,438.292740,10.755840,106.668792,236.869,15,26.0
5,quận,6,7.1900,258945.0,36.014604,10.745512,106.617478,113.000,14,41.0
6,quận,7,35.6900,324620.0,9.095545,10.737741,106.645937,97.999,10,1.0
7,quận,8,19.1800,451290.0,23.529197,10.723041,106.562972,84.500,18,40.0
8,quận,9,114.0000,397000.0,3.482456,10.821292,106.594480,41.800,13,15.0
9,quận,10,5.7200,372450.0,65.113636,10.772759,106.648828,237.000,15,23.0


In [65]:
del population_df['average_price']

In [66]:
population_df = population_df.dropna(subset = ['district'])

In [67]:
population_df = population_df.sort_values(by = 'district')
population_df

,administrative_genre,name,acreage,population,density,lat,lon,num_of_ward,district
13,quận,thủ đức,48.0000,524670.0,10.930625,10.820455,106.625025,12,0.0
6,quận,7,35.6900,324620.0,9.095545,10.737741,106.645937,10,1.0
9,quận,nam từ liêm,32.2736,232894.0,7234.000000,21.019617,105.720391,10,2.0
5,quận,hai bà trưng,9.6000,284615.0,29.647000,21.005170,105.841241,23,3.0
27,huyện,sóc sơn,306.5000,300000.0,978.000000,21.281270,105.666126,21,4.0
17,huyện,đan phượng,76.7400,142068.0,1851.000000,21.118224,105.599403,16,4.0
2,quận,cầu giấy,12.0000,225600.0,18.800000,21.028754,105.749714,8,5.0
8,quận,long biên,60.4000,226900.0,3757.000000,21.037201,105.807726,14,6.0
3,quận,đống đa,10.2000,370100.0,36.284000,21.014746,105.799923,21,7.0
21,huyện,hoài đức,84.9300,276070.0,3250.000000,21.022977,105.611171,20,8.0


In [68]:
population_df[['administrative_genre', 'district']]

,administrative_genre,district
13,quận,0.0
6,quận,1.0
9,quận,2.0
5,quận,3.0
27,huyện,4.0
17,huyện,4.0
2,quận,5.0
8,quận,6.0
3,quận,7.0
21,huyện,8.0


In [69]:
ad_df = population_df[['administrative_genre', 'district']].drop_duplicates(subset = ['district'], keep = 'first')
ad_df

,administrative_genre,district
13,quận,0.0
6,quận,1.0
9,quận,2.0
5,quận,3.0
27,huyện,4.0
2,quận,5.0
8,quận,6.0
3,quận,7.0
21,huyện,8.0
19,huyện,9.0


In [70]:
population_df = population_df.groupby('district')[['acreage', 'population', 'density', 'lat', 'lon', 'num_of_ward']].mean().reset_index()

In [71]:
population_df = ad_df.merge(population_df, how='left', on = ['district'])
population_df

,administrative_genre,district,acreage,population,density,lat,lon,num_of_ward
0,quận,0.0,48.000000,5.246700e+05,10.930625,10.820455,106.625025,12.000000
1,quận,1.0,35.690000,3.246200e+05,9.095545,10.737741,106.645937,10.000000
2,quận,2.0,32.273600,2.328940e+05,7234.000000,21.019617,105.720391,10.000000
3,quận,3.0,9.600000,2.846150e+05,29.647000,21.005170,105.841241,23.000000
4,huyện,4.0,191.620000,2.210340e+05,1414.500000,21.199747,105.632765,18.500000
5,quận,5.0,12.000000,2.256000e+05,18.800000,21.028754,105.749714,8.000000
6,quận,6.0,60.400000,2.269000e+05,3757.000000,21.037201,105.807726,14.000000
7,quận,7.0,10.200000,3.701000e+05,36.284000,21.014746,105.799923,21.000000
8,huyện,8.0,84.930000,2.760700e+05,3250.000000,21.022977,105.611171,20.000000
9,huyện,9.0,114.790000,2.439570e+05,2125.000000,21.022715,105.784723,22.000000


In [72]:
ad_list = list(population_df['administrative_genre'].unique())
ad_dict = dict(zip(ad_list, range(len(ad_list))))
ad_dict

{'quận': 0, 'huyện': 1}

In [73]:
population_df['administrative_genre'] = population_df['administrative_genre'].map(ad_dict)
population_df = population_df.rename(columns = {'lat': 'district_lat', 'lon': 'district_lon'})

In [74]:
df = df.merge(population_df, how='left', on = 'district')
df

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,num_of_place_of_worship_in_2000m_radius,num_of_bank_in_2000m_radius,num_of_atm_in_2000m_radius,administrative_genre,acreage,population,density,district_lat,district_lon,num_of_ward
0,1,2.0,2.0,NaN,NaN,0,0,0,0,0,...,20,3,3,0,48.0000,524670.0,10.930625,10.820455,106.625025,12.0
1,1,2.0,3.0,NaN,NaN,1,1,1,1,0,...,3,12,20,0,35.6900,324620.0,9.095545,10.737741,106.645937,10.0
2,1,NaN,NaN,NaN,NaN,1,2,2,2,1,...,7,2,11,0,32.2736,232894.0,7234.000000,21.019617,105.720391,10.0
3,4,NaN,NaN,NaN,NaN,0,3,3,3,1,...,4,13,59,0,9.6000,284615.0,29.647000,21.005170,105.841241,23.0
4,1,NaN,NaN,NaN,NaN,1,4,4,4,1,...,0,0,0,1,191.6200,221034.0,1414.500000,21.199747,105.632765,18.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170959,5,3.0,4.0,NaN,NaN,1,39,240,5,1,...,24,37,122,0,12.0000,225600.0,18.800000,21.028754,105.749714,8.0
170960,1,1.0,1.0,NaN,NaN,0,44,411,5,1,...,14,39,92,0,12.0000,225600.0,18.800000,21.028754,105.749714,8.0
170961,5,4.0,4.0,NaN,NaN,1,35,39,7,1,...,73,105,354,0,10.2000,370100.0,36.284000,21.014746,105.799923,21.0
170962,1,NaN,NaN,NaN,NaN,0,45,50,19,1,...,4,4,6,0,24.0000,130600.0,5442.000000,21.069269,105.779534,8.0


In [75]:
del df['numberOfKitchens']

In [76]:
del df['numberOfGarages']

In [77]:
city_dict

{'hồ chí minh': 0, 'hà nội': 1}

In [78]:
best_df1 = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_famous_place.csv')
best_df1['city'] = 'hồ chí minh'

best_df2 = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hn_famous_place.csv')
best_df2['city'] = 'hà nội'

best_df = pd.concat([best_df1, best_df2])
best_df

,lat,lon,name,address,city
0,10.772657,106.698129,TTTM Saigon Centre (Takashimaya),"92 – 94 Nam Kỳ Khởi Nghĩa, P. Bến Nghé, Quận 1...",hồ chí minh
1,10.727873,106.717026,Crescent Mall,"101 Tôn Dật Tiên, P. Tân Phú, Quận 7, TP. HCM",hồ chí minh
2,10.727873,106.717026,SC VivoCity Shopping Center,"1058 Nguyễn Văn Linh, KP. 1, P. Tân Phong, Quậ...",hồ chí minh
3,10.729980,106.700595,Aeon Mall Shopping Center Tân Phú,"30 Bờ Bao Tân Thắng, P. Sơn Kỳ, Quận Tân Phú, ...",hồ chí minh
4,10.801344,106.615384,Saigon Garden,"99 Nguyễn Huệ , Quận 1, TP. HCM",hồ chí minh
...,...,...,...,...,...
47,21.026225,105.843441,Phố Hai Bà Trưng,NaN,hà nội
48,21.024363,105.844308,Phố Lý Thường Kiệt,NaN,hà nội
49,21.013373,105.846550,Phố Bà Triệu,NaN,hà nội
50,21.021383,105.846880,Hàng Bài,NaN,hà nội


In [79]:
del best_df['address']
best_df['city'] = best_df['city'].map(city_dict)

In [80]:
import math
from tqdm import tqdm
def distance_func(lat1: float, lon1: float, lat2: float, lon2: float):

    try:
        R = 6371
        dLat = (lat2-lat1) * math.pi / 180
        dLon = (lon2-lon1) * math.pi / 180
        lat1 = lat1 * math.pi / 180
        lat2 = lat2 * math.pi / 180
        a = math.sin(dLat/2) * math.sin(dLat/2) + math.sin(dLon/2) * \
            math.sin(dLon/2) * math.cos(lat1) * math.cos(lat2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = R * c
        return d*1000
    except:
        print(f"Distance_Func error with: {(lat1, lon1, lat2, lon2)}")

def distance(lat1, lon1, lat2, lon2, city1, city2):
    if city1 != city2:
        return -1
    return distance_func(lat1, lon1, lat2, lon2)


best_arr = best_df.values
for i in tqdm(range(len(best_arr))):
    lat, lon = best_arr[i][0], best_arr[i][1]
    name = best_arr[i][2]
    city = 'hn' if best_arr[i][3] == 0 else 'hcm'

    df[f'distance_{city}_{name}'] = df.apply(lambda x: distance(x['lat'], x['lon'], lat, lon, x['city'], best_arr[i][3]), axis = 1)


 84%|████████▎ | 87/104 [04:03<00:53,  3.13s/it]/tmp/ipykernel_1026444/2436538999.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'distance_{city}_{name}'] = df.apply(lambda x: distance(x['lat'], x['lon'], lat, lon, x['city'], best_arr[i][3]), axis = 1)
 85%|████████▍ | 88/104 [04:07<00:50,  3.14s/it]/tmp/ipykernel_1026444/2436538999.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'distance_{city}_{name}'] = df.apply(lambda x: distance(x['lat'], x['lon'], lat, lon, x['city'], best_arr[i][3]), axis = 1)
 86%|███

In [81]:
df['distance_district'] = df.apply(lambda x: distance_func(x['lat'], x['lon'], x['district_lat'], x['district_lon']), axis = 1)

/tmp/ipykernel_1026444/3795960773.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['distance_district'] = df.apply(lambda x: distance_func(x['lat'], x['lon'], x['district_lat'], x['district_lon']), axis = 1)


In [82]:
distance_cols = [c for c in df.columns.tolist() if 'distance' in c]
for c in distance_cols:
    df[c] = np.log(df[c])


/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [83]:
df['target'] = df['price'] * 1000 / df['landSize']

/tmp/ipykernel_1026444/343422437.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['target'] = df['price'] * 1000 / df['landSize']


In [84]:
df['population'] = np.log(df['population'])

In [85]:
df['density'] = np.log(df['density'])

In [86]:
df['acreage'] = np.log(df['acreage'])

In [87]:
df

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,certificateOfLandUseRight,ward,street,district,city,lat,lon,...,distance_hcm_Hàng Khay,distance_hcm_Hàng Trống,distance_hcm_Hàng Hành,distance_hcm_Phố Hai Bà Trưng,distance_hcm_Phố Lý Thường Kiệt,distance_hcm_Phố Bà Triệu,distance_hcm_Hàng Bài,distance_hcm_Phố Hai Bà Trưng(Hoàn Kiếm),distance_district,target
0,1,2.0,2.0,0,0,0,0,0,10.860736,106.739182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.491548,25.862069
1,1,2.0,3.0,1,1,1,1,0,10.722855,106.714168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.940605,56.666667
2,1,NaN,NaN,1,2,2,2,1,21.008299,105.774104,...,8.991444,8.953072,8.991857,8.918372,8.923035,8.928135,8.948050,8.962571,8.650960,0.000000
3,4,NaN,NaN,0,3,3,3,1,20.998322,105.867558,...,8.191673,8.314504,8.346112,8.290699,8.234694,7.918967,8.114929,8.206480,7.950181,109.285714
4,1,NaN,NaN,1,4,4,4,1,21.357386,105.823221,...,10.517558,10.508597,10.500917,10.515509,10.521235,10.553958,10.530557,10.519285,10.180923,1.944444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170959,5,3.0,4.0,1,39,240,5,1,21.023187,105.794927,...,8.637401,8.569255,8.618378,8.526488,8.542299,8.606724,8.593435,8.597713,8.462399,152.857143
170960,1,1.0,1.0,0,44,411,5,1,21.008718,105.797279,...,8.650876,8.604715,8.663349,8.550972,8.552994,8.544912,8.582529,8.609808,8.597210,47.368421
170961,5,4.0,4.0,1,35,39,7,1,21.023378,105.832225,...,7.485171,7.300310,7.503547,7.095394,7.138039,7.526720,7.337702,7.350129,8.156931,197.169811
170962,1,NaN,NaN,0,45,50,19,1,21.089607,105.812688,...,8.991347,8.933380,8.912594,8.953719,8.982472,9.124377,9.032998,8.987765,8.322794,131.868132


In [88]:
df.to_csv('../data/process_v1/process_data_5.csv', index = False)